# <span style="color: skyblue; ">Ch07 Dash コールバック</span>


## <span style="color: skyblue; ">7.1 コールバック基礎</span>

- コールバックの概要
- 入力データを即座に反映するコールバック
- 複数の入出力が存在するコールバック

### <span style="color: skyblue; ">7.1.1 コールバックの概要</span>

In [1]:
# ボタンをクリックするとテキストエリアへ入力された文字を見出しに描画する

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# レイアウト
def server_layout():
    return html.Div(
        [
            # コールバックの返り値を表示する
            html.H1(id="output-header-title"),
            html.P("タイトルをテキストエリア値に変更する"),
            # 文字列を入力するテキストエリア
            dcc.Textarea(
                id="input-text-state",
                value="initial value",
                style=dict(width="80%", fontSize="30px"),
            ),
            # クリックするとコールバックを呼び出すボタン
            html.Button(
                id="submit-button",
                n_clicks=0,
                children="Submit"
            ),
        ],
        style=dict(margin=50),
    )
app.layout = server_layout

# コールバックの作成
@app.callback(
    Output(component_id="output-header-title", component_property="children"),
    Input(component_id="submit-button", component_property="n_clicks"),
    State("input-text-state", "value"),
)
def update_title(n_click, text_value):
    return text_value


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.1.2 入力データを即座に反映するコールバック</span>

- [dcc.Slider](https://dash.plotly.com/dash-core-components/slider)


In [1]:
# スライダのハンドル位置の変化をH1に反映する
# updatemode="drag": 動作を即座に反映させる

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div(
        [
            html.H1(id="callback-output"),
            dcc.Slider(
                id="callback-input",
                min=0,
                max=100,
                value=0,
                marks=None,
                updatemode="drag"
            ),
        ],
        style=dict(textAlign="center", width="60%", margin="auto"),
    )
app.layout = server_layout

# コールバックの作成
@app.callback(
    Output(component_id="callback-output", component_property="children"),
    Input(component_id="callback-input", component_property="value"),
)
def update_title(num_value):
    return num_value


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.1.3 複数の入出力が存在するコールバック</span>

- tips データセットを用いて、コールバックに出力項目と入力項目を2つずつもつアプリケーション
- 2つのドロップダウンともつ
  - 一方は表示する曜日の選択
  - 一方は表示するグラフの選択
  - 棒グラフには曜日ごとの売上の累計
  - 散布図にはX軸に会計額、Y軸にチップ金額を表示

In [1]:
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

tips = px.data.tips()

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# レイアウト
def server_layout():
    return html.Div(
        [
            html.H3(id="title", style=dict(textAlign="center")),
            html.Div(
                [
                    html.Div(
                        [
                            html.H4("曜日選択"),
                            dcc.Dropdown(
                                id="day_selector",
                                options=[
                                    dict(label=dow, value=dow) for dow in tips.day.unique()
                                ],
                                # 複数選択可
                                multi=True,
                                value=["Thur", "Fri", "Sat", "Sun"],
                            ),
                        ],
                        className="six columns",
                    ),
                    html.Div(
                        [
                            html.H4("グラフ選択"),
                            dcc.Dropdown(
                                id="graph_selector",
                                options=[
                                    dict(label="bar", value="bar"),
                                    dict(label="scatter", value="scatter"),
                                ],
                                value="bar",
                            ),
                        ],
                        className="six columns",
                    ),
                ],
                style=dict(padding="2%", margin="auto"),
            ),
            dcc.Graph(id="app_graph", style=dict(padding="5%", marginTop="50px")),
        ],
    )
app.layout = server_layout

# コールバックの作成
@app.callback(
    Output(component_id="title", component_property="children"),
    Output(component_id="app_graph", component_property="figure"),
    Input(component_id="day_selector", component_property="value"),
    Input(component_id="graph_selector", component_property="value"),
)
def update_graph(selected_days, selected_graph):
    selected_df = tips[tips["day"].isin(selected_days)]
    if selected_graph == "scatter":
        title = "テーブルごとのデータ（散布図）"
        figure = px.scatter(
            selected_df, x="total_bill", y="tip", color="smoker", height=600
        )
    else:
        title = "曜日ごとの売上（棒グラフ）",
        figure = px.bar(selected_df, x="day", y="total_bill", height=600)
    return title, figure


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
